# Understanding Bayes Nets

We have:

- Nodes: Variables (features like Age, Stress_Level and Mental_Health_Condition)
- Edges: Dependencies between variables (Stress_Level -> Mental_Health_Condition)

Bayes Nets encode the joint probability distribution of variables, allowing inference given evidence.

# Setup

In [356]:
from inspect import getsource
from IPython.display import display
import pandas as pd
import numpy as np
from collections import defaultdict, Counter
import itertools
import math
import random
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from matplotlib import pyplot

In [357]:
random.seed(42)

In [358]:
df = pd.read_csv('../data/data_covid.csv')

In [359]:
df.replace({97: pd.NA, 99: pd.NA}, inplace=True)

In [360]:
categorical_columns = df.select_dtypes(include=['object', 'category']).columns

for col in categorical_columns:
    # Check if the column contains missing values
    if df[col].isna().any():
        # Replace missing values with the mode (most frequent value)
        mode_value = df[col].mode()[0]
        df[col] = df[col].fillna(mode_value)  # Reassigning directly to avoid chained assignment warning

# Confirm the changes
print(df[categorical_columns].head())

/var/folders/tg/nw46x5hx3x3_5gqfd9j7zhcc0000gn/T/ipykernel_2790/3004967662.py:8: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].fillna(mode_value)  # Reassigning directly to avoid chained assignment warning


    DATE_DIED  INTUBED  PNEUMONIA  AGE  PREGNANT  ICU
0  03/05/2020        2          1   65         2    2
1  03/06/2020        2          1   72         2    2
2  09/06/2020        1          2   55         2    2
3  12/06/2020        2          2   53         2    2
4  21/06/2020        2          2   68         2    2


In [361]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 21 columns):
 #   Column                Non-Null Count    Dtype 
---  ------                --------------    ----- 
 0   USMER                 1048575 non-null  int64 
 1   MEDICAL_UNIT          1048575 non-null  int64 
 2   SEX                   1048575 non-null  int64 
 3   PATIENT_TYPE          1048575 non-null  int64 
 4   DATE_DIED             1048575 non-null  object
 5   INTUBED               1048575 non-null  int64 
 6   PNEUMONIA             1048575 non-null  int64 
 7   AGE                   1048575 non-null  int64 
 8   PREGNANT              1048575 non-null  int64 
 9   DIABETES              1048575 non-null  int64 
 10  COPD                  1048575 non-null  int64 
 11  ASTHMA                1048575 non-null  int64 
 12  INMSUPR               1048575 non-null  int64 
 13  HIPERTENSION          1048575 non-null  int64 
 14  OTHER_DISEASE         1048575 non-null  int64 
 15

In [362]:
age_bins = [0, 20, 40, 60, 80, 120] 
age_labels = ['<20', '20-40', '40-60', '60-80', '80+'] 

df['AGE_GROUP'] = pd.cut(df['AGE'], bins=age_bins, labels=age_labels, right=False)

print(df[['AGE', 'AGE_GROUP']].head())

   AGE AGE_GROUP
0   65     60-80
1   72     60-80
2   55     40-60
3   53     40-60
4   68     60-80


In [363]:
df['CLASIFFICATION_FINAL'] = df['CLASIFFICATION_FINAL'].apply(lambda x: 1 if x in [1, 2, 3] else 0)

print(df['CLASIFFICATION_FINAL'].value_counts())

CLASIFFICATION_FINAL
0    656596
1    391979
Name: count, dtype: int64


In [364]:
# check if there is nan values in the AGE_GROUP column
print(df['AGE_GROUP'].isnull().sum())
df['AGE_GROUP'].fillna(df['AGE_GROUP'].mode()[0], inplace=True)

6


/var/folders/tg/nw46x5hx3x3_5gqfd9j7zhcc0000gn/T/ipykernel_2790/1712841479.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['AGE_GROUP'].fillna(df['AGE_GROUP'].mode()[0], inplace=True)


In [365]:
train_data, test_data = train_test_split(
    df,
    test_size=0.2,
    random_state=42,
    stratify=df["CLASIFFICATION_FINAL"]
)

In [366]:
categorical_columns = ['AGE_GROUP', 'PNEUMONIA', 'ICU', 'CLASIFFICATION_FINAL', 'SEX', 'OBESITY', 'DIABETES']
for col in categorical_columns:
    train_data[col] = train_data[col].astype("category")
    test_data[col] = test_data[col].astype("category")

# Helper Code

In [367]:
def extend(s, var, val):
    """Copy dict s and extend it by setting var to val; return copy."""
    return {**s, var: val}

In [368]:
def consistent_with(event, evidence):
    """Is event consistent with the given evidence?"""
    return all(evidence.get(k, v) == v
               for k, v in event.items())

In [369]:
def probability(p):
    """Return true with probability p."""
    return p > random.uniform(0.0, 1.0)

In [370]:
class ProbDist:
    """A discrete probability distribution. You name the random variable
    in the constructor, then assign and query probability of values.
    >>> P = ProbDist('Flip'); P['H'], P['T'] = 0.25, 0.75; P['H']
    0.25
    >>> P = ProbDist('X', {'lo': 125, 'med': 375, 'hi': 500})
    >>> P['lo'], P['med'], P['hi']
    (0.125, 0.375, 0.5)
    """

    def __init__(self, varname='?', freqs=None):
        """If freqs is given, it is a dictionary of values - frequency pairs,
        then ProbDist is normalized."""
        self.prob = {}
        self.varname = varname
        self.values = []
        if freqs:
            for (v, p) in freqs.items():
                self[v] = p
            self.normalize()

    def __getitem__(self, val):
        """Given a value, return P(value)."""
        try:
            return self.prob[val]
        except KeyError:
            return 0

    def __setitem__(self, val, p):
        """Set P(val) = p."""
        if val not in self.values:
            self.values.append(val)
        self.prob[val] = p

    def normalize(self):
        """Make sure the probabilities of all values sum to 1.
        Returns the normalized distribution.
        Raises a ZeroDivisionError if the sum of the values is 0.""" 
        total = sum(self.prob.values())
        if not np.isclose(total, 1.0):
            for val in self.prob:
                self.prob[val] /= total
        return self

    def show_approx(self, numfmt='{:.3g}'):
        """Show the probabilities rounded and sorted by key, for the
        sake of portable doctests."""
        return ', '.join([('{}: ' + numfmt).format(v, p)
                          for (v, p) in sorted(self.prob.items())])

    def __repr__(self):
        return "P({})".format(self.varname)

In [371]:
class JointProbDist(ProbDist):
    """A discrete probability distribute over a set of variables.
    >>> P = JointProbDist(['X', 'Y']); P[1, 1] = 0.25
    >>> P[1, 1]
    0.25
    >>> P[dict(X=0, Y=1)] = 0.5
    >>> P[dict(X=0, Y=1)]
    0.5"""

    def __init__(self, variables):
        self.prob = {}
        self.variables = variables
        self.vals = defaultdict(list)

    def __getitem__(self, values):
        """Given a tuple or dict of values, return P(values)."""
        values = event_values(values, self.variables)
        return ProbDist.__getitem__(self, values)

    def __setitem__(self, values, p):
        """Set P(values) = p.  Values can be a tuple or a dict; it must
        have a value for each of the variables in the joint. Also keep track
        of the values we have seen so far for each variable."""
        values = event_values(values, self.variables)
        self.prob[values] = p
        for var, val in zip(self.variables, values):
            if val not in self.vals[var]:
                self.vals[var].append(val)

    def values(self, var):
        """Return the set of possible values for a variable."""
        return self.vals[var]

    def __repr__(self):
        return "P({})".format(self.variables)

In [372]:
class BayesNode:
    """A conditional probability distribution for a boolean variable,
    P(X | parents). Part of a BayesNet."""

    def __init__(self, X, parents, cpt):
        """X is a variable name, and parents a sequence of variable
        names or a space-separated string.  cpt, the conditional
        probability table, takes one of these forms:

        * A number, the unconditional probability P(X=true). You can
          use this form when there are no parents.

        * A dict {v: p, ...}, the conditional probability distribution
          P(X=true | parent=v) = p. When there's just one parent.

        * A dict {(v1, v2, ...): p, ...}, the distribution P(X=true |
          parent1=v1, parent2=v2, ...) = p. Each key must have as many
          values as there are parents. You can use this form always;
          the first two are just conveniences.

        In all cases the probability of X being false is left implicit,
        since it follows from P(X=true).

        >>> X = BayesNode('X', '', 0.2)
        >>> Y = BayesNode('Y', 'P', {T: 0.2, F: 0.7})
        >>> Z = BayesNode('Z', 'P Q',
        ...    {(T, T): 0.2, (T, F): 0.3, (F, T): 0.5, (F, F): 0.7})
        """
        if isinstance(parents, str):
            parents = parents.split()

        # We store the table always in the third form above.
        if isinstance(cpt, (float, int)):  # no parents, 0-tuple
            cpt = {(): cpt}
        elif isinstance(cpt, dict):
            # one parent, 1-tuple
            if cpt and isinstance(list(cpt.keys())[0], bool):
                cpt = {(v,): p for v, p in cpt.items()}

        assert isinstance(cpt, dict)
        for vs, p in cpt.items():
            assert isinstance(vs, tuple) and len(vs) == len(parents)
            assert all(isinstance(v, bool) for v in vs)
            assert 0 <= p <= 1

        self.variable = X
        self.parents = parents
        self.cpt = cpt
        self.children = []

    def p(self, value, event):
        """Return the conditional probability
        P(X=value | parents=parent_values), where parent_values
        are the values of parents in event. (event must assign each
        parent a value.)
        >>> bn = BayesNode('X', 'Burglary', {T: 0.2, F: 0.625})
        >>> bn.p(False, {'Burglary': False, 'Earthquake': True})
        0.375"""
        assert isinstance(value, bool)
        ptrue = self.cpt[event_values(event, self.parents)]
        return ptrue if value else 1 - ptrue

    def sample(self, event):
        """Sample from the distribution for this variable conditioned
        on event's values for parent_variables. That is, return True/False
        at random according with the conditional probability given the
        parents."""
        return probability(self.p(True, event))

    def __repr__(self):
        return repr((self.variable, ' '.join(self.parents)))

In [373]:
def probability_sampling(probabilities):
    """Randomly sample an outcome from a probability distribution."""
    total = sum(probabilities.values())
    r = random.uniform(0, total)
    cumulative = 0
    for outcome, prob in probabilities.items():
        cumulative += prob
        if r <= cumulative:
            return outcome
    return None  # Should not reach here if probabilities are normalize

In [374]:
class MultiClassBayesNode:
    """A Bayesian node for multi-class variables."""
    def __init__(self, X, parents, cpt):
        """
        X: Name of the variable.
        parents: List of parent variable names.
        cpt: Conditional probability table, mapping tuples of parent values to
             dictionaries of target probabilities.
        """
        if isinstance(parents, str):
            parents = parents.split()
        self.variable = X
        self.parents = parents
        self.cpt = cpt
        self.children = []

    def p(self, value, event):
        """Return the conditional probability P(X=value | parents=parent_values)."""
        parent_values = tuple(event.get(p, None) for p in self.parents)
        probabilities = self.cpt.get(parent_values, {})
        return probabilities.get(value, 0)  # Default to 0 if value not found

    def sample(self, event):
        """Sample from the distribution for this variable given parent values."""
        parent_values = tuple(event.get(p, None) for p in self.parents)
        probabilities = self.cpt.get(parent_values, {})
        return probability_sampling(probabilities)

    def __repr__(self):
        return repr((self.variable, ' '.join(self.parents)))

In [375]:
class BayesNet:
    """Bayesian network containing only boolean-variable or multi-class nodes."""
    
    def __init__(self, node_specs=None):
        """Nodes must be ordered with parents before children."""
        self.nodes = []
        self.variables = []
        node_specs = node_specs or []
        for node_spec in node_specs:
            self.add(node_spec)

    def add(self, node_spec):
        """Add a node to the net. Supports both pre-constructed nodes and node specs."""
        if isinstance(node_spec, (BayesNode, MultiClassBayesNode)):
            # If already a node, add it directly
            node = node_spec
        else:
            # Otherwise, initialize a new node
            node = BayesNode(*node_spec)

        assert node.variable not in self.variables
        assert all((parent in self.variables) for parent in node.parents)

        self.nodes.append(node)
        self.variables.append(node.variable)

        # Register children for the parent nodes
        for parent in node.parents:
            self.variable_node(parent).children.append(node)

    def variable_node(self, var):
        """Return the node for the variable named var."""
        for n in self.nodes:
            if n.variable == var:
                return n
        raise Exception(f"No such variable: {var}")

    def variable_values(self, var):
        """Return the domain of var."""
        return [True, False]

    def __repr__(self):
        return f"BayesNet({self.nodes!r})"

In [376]:
def make_factor(var, e, bn):
    """Return the factor for var in bn's joint distribution given e.
    That is, bn's full joint distribution, projected to accord with e,
    is the pointwise product of these factors for bn's variables."""
    node = bn.variable_node(var)
    variables = [X for X in [var] + node.parents if X not in e]
    cpt = {event_values(e1, variables): node.p(e1[var], e1)
           for e1 in all_events(variables, bn, e)}
    return Factor(variables, cpt)

In [377]:
class Factor:
    """A factor in a joint distribution."""

    def __init__(self, variables, cpt):
        self.variables = variables
        self.cpt = cpt

    def pointwise_product(self, other, bn):
        """Multiply two factors, combining their variables."""
        variables = list(set(self.variables) | set(other.variables))
        cpt = {event_values(e, variables): self.p(e) * other.p(e) for e in all_events(variables, bn, {})}
        return Factor(variables, cpt)

    def sum_out(self, var, bn):
        """Make a factor eliminating var by summing over its values."""
        variables = [X for X in self.variables if X != var]
        cpt = {event_values(e, variables): sum(self.p(extend(e, var, val)) for val in bn.variable_values(var))
               for e in all_events(variables, bn, {})}
        return Factor(variables, cpt)

    def normalize(self):
        """Return my probabilities; must be down to one variable."""
        assert len(self.variables) == 1
        return ProbDist(self.variables[0], {k: v for ((k,), v) in self.cpt.items()})

    def p(self, e):
        """Look up my value tabulated for e."""
        return self.cpt[event_values(e, self.variables)]

In [378]:
def enumerate_all(variables, e, bn):
    """Return the sum of those entries in P(variables | e{others})
    consistent with e, where P is the joint distribution represented
    by bn, and e{others} means e restricted to bn's other variables
    (the ones other than variables). Parents must precede children in variables."""
    if not variables:
        return 1.0
    Y, rest = variables[0], variables[1:]
    Ynode = bn.variable_node(Y)
    if Y in e:
        return Ynode.p(e[Y], e) * enumerate_all(rest, e, bn)
    else:
        return sum(Ynode.p(y, e) * enumerate_all(rest, extend(e, Y, y), bn)
                   for y in bn.variable_values(Y))

In [379]:
def enumeration_ask(X, e, bn):
    """Return the conditional probability distribution of variable X
    given evidence e, from BayesNet bn. [Figure 14.9]
    >>> enumeration_ask('Burglary', dict(JohnCalls=T, MaryCalls=T), burglary
    ...  ).show_approx()
    'False: 0.716, True: 0.284'"""
    assert X not in e, "Query variable must be distinct from evidence"
    Q = ProbDist(X)
    for xi in bn.variable_values(X):
        Q[xi] = enumerate_all(bn.variables, extend(e, X, xi), bn)
    return Q.normalize()

In [380]:
def elimination_ask(X, e, bn):
    """Compute bn's P(X|e) by variable elimination. [Figure 14.11]
    >>> elimination_ask('Burglary', dict(JohnCalls=T, MaryCalls=T), burglary
    ...  ).show_approx()
    'False: 0.716, True: 0.284'"""
    assert X not in e, "Query variable must be distinct from evidence"
    factors = []
    for var in reversed(bn.variables):
        factors.append(make_factor(var, e, bn))
        if is_hidden(var, X, e):
            factors = sum_out(var, factors, bn)
    return pointwise_product(factors, bn).normalize()

In [381]:
def prior_sample(bn):
    """Randomly sample from bn's full joint distribution. The result
    is a {variable: value} dict. [Figure 14.13]"""
    event = {}
    for node in bn.nodes:
        event[node.variable] = node.sample(event)
    return event

In [382]:
def rejection_sampling(X, e, bn, N=10000):
    """Estimate the probability distribution of variable X given
    evidence e in BayesNet bn, using N samples.  [Figure 14.14]
    Raises a ZeroDivisionError if all the N samples are rejected,
    i.e., inconsistent with e.
    >>> random.seed(47)
    >>> rejection_sampling('Burglary', dict(JohnCalls=T, MaryCalls=T),
    ...   burglary, 10000).show_approx()
    'False: 0.7, True: 0.3'
    """
    counts = {x: 0 for x in bn.variable_values(X)}  # bold N in [Figure 14.14]
    for j in range(N):
        sample = prior_sample(bn)  # boldface x in [Figure 14.14]
        if consistent_with(sample, e):
            counts[sample[X]] += 1
    return ProbDist(X, counts)

In [383]:
def likelihood_weighting(X, e, bn, N=10000):
    """Estimate the probability distribution of variable X given
    evidence e in BayesNet bn.  [Figure 14.15]
    >>> random.seed(1017)
    >>> likelihood_weighting('Burglary', dict(JohnCalls=T, MaryCalls=T),
    ...   burglary, 10000).show_approx()
    'False: 0.702, True: 0.298'
    """
    W = {x: 0 for x in bn.variable_values(X)}
    for j in range(N):
        sample, weight = weighted_sample(bn, e)  # boldface x, w in [Figure 14.15]
        W[sample[X]] += weight
    return ProbDist(X, W)

In [384]:
def gibbs_ask(X, e, bn, N=1000):
    """[Figure 14.16]"""
    assert X not in e, "Query variable must be distinct from evidence"
    counts = {x: 0 for x in bn.variable_values(X)}  # bold N in [Figure 14.16]
    Z = [var for var in bn.variables if var not in e]
    state = dict(e)  # boldface x in [Figure 14.16]
    for Zi in Z:
        state[Zi] = random.choice(bn.variable_values(Zi))
    for j in range(N):
        for Zi in Z:
            state[Zi] = markov_blanket_sample(Zi, state, bn)
            counts[state[X]] += 1
    return ProbDist(X, counts)

In [385]:
def pointwise_product(self, other, bn):
        """Multiply two factors, combining their variables."""
        variables = list(set(self.variables) | set(other.variables))
        cpt = {event_values(e, variables): self.p(e) * other.p(e)
               for e in all_events(variables, bn, {})}
        return Factor(variables, cpt)

In [386]:
def sum_out(self, var, bn):
        """Make a factor eliminating var by summing over its values."""
        variables = [X for X in self.variables if X != var]
        cpt = {event_values(e, variables): sum(self.p(extend(e, var, val))
                                               for val in bn.variable_values(var))
               for e in all_events(variables, bn, {})}
        return Factor(variables, cpt)

In [387]:
def event_values(event, variables):
    """Return a tuple of the values of variables in event.
    >>> event_values ({'A': 10, 'B': 9, 'C': 8}, ['C', 'A'])
    (8, 10)
    >>> event_values ((1, 2), ['C', 'A'])
    (1, 2)
    """
    if isinstance(event, tuple) and len(event) == len(variables):
        return event
    else:
        return tuple([event[var] for var in variables])

# Design the Network Structure

Find dependencies.

Mental_Health_Condition is the target variable.

It depends on:
- Gender, Physical Activity Hours.

Other variables are conditionally dependent based on logical relationships:
- Physical Activity Hours depend on:
    - Country
- Gender depends on:
    - Occupation
- Country depends on:
    - Stress Level
- Stress Level depends on:
    - Age

# Estimate Conditional Probabilities

If a node is a root node, then estimate probability directly from the data. Estimate conditional probabilities based on parent for non-root nodes.

In [388]:
def compute_cpt(data, target, parents, alpha=1):
    """
    Compute CPT with Laplace smoothing.
    
    Args:
        data: pandas DataFrame (training data)
        target: str, target variable
        parents: list of parent variable names
        alpha: smoothing parameter (default=1)
    
    Returns:
        cpt: dict { parent_values_tuple: { target_value: probability } }
    """
    target_values = data[target].cat.categories

    if not parents:
        # Marginal distribution of target
        counts = defaultdict(lambda: alpha)
        for val in data[target]:
            counts[val] += 1
        total = sum(counts.values())
        cpt = {(): {tv: counts[tv]/total for tv in counts}}
        return cpt

    # Determine possible parent combinations
    from itertools import product
    parent_values_list = [data[p].cat.categories for p in parents]
    parent_combinations = list(product(*parent_values_list)) if parents else [()]

    # Initialize counts with alpha
    counts = {pc: defaultdict(lambda: alpha) for pc in parent_combinations}

    # Count occurrences
    for _, row in data.iterrows():
        pv = tuple(row[p] for p in parents) if parents else ()
        tv = row[target]
        counts[pv][tv] += 1

    # Compute probabilities
    cpt = {}
    for pc in parent_combinations:
        total = sum(counts[pc].values())
        cpt[pc] = {tv: (counts[pc][tv] / total) for tv in counts[pc]}
        
    return cpt

In [389]:
cpt_classification = compute_cpt(train_data, 'CLASIFFICATION_FINAL', ['SEX', 'OBESITY', 'DIABETES', 'ICU'])

In [390]:
cpt_icu = compute_cpt(train_data, 'ICU', ['PNEUMONIA'])

In [391]:
cpt_pneumonia = compute_cpt(train_data, 'PNEUMONIA', ['AGE_GROUP'])

In [392]:
# calculate cpt for the ones which have no parents (AGE_GROUP), (SEX), (OBESITY), (DIABETES)
cpt_age_group = compute_cpt(train_data, 'AGE_GROUP', [])
cpt_sex_group = compute_cpt(train_data, 'SEX', [])
cpt_obesity_group = compute_cpt(train_data, 'OBESITY', [])
cpt_diabetes_group = compute_cpt(train_data, 'DIABETES', [])

# Implement Inference

Use the chain rule of probability to predict target given evidence.

In [393]:
classification_node = MultiClassBayesNode(
    "CLASIFFICATION_FINAL",
    ['SEX', 'OBESITY', 'DIABETES', 'ICU'],
    cpt_classification
)

In [394]:
pneumonia_node = MultiClassBayesNode(
    "PNEUMONIA", 
    ["AGE_GROUP"],
    cpt_pneumonia
)

In [395]:
icu_node = MultiClassBayesNode(
    "ICU", 
    ["PNEUMONIA"], 
    cpt_icu
)

In [396]:
age_node = MultiClassBayesNode(
    "AGE_GROUP",
    [],
    cpt_age_group
)

In [397]:
# create nodes for the ones which have no parents
obesity_node = MultiClassBayesNode(
    "OBESITY",
    [],
    cpt_obesity_group
)

In [398]:
sex_node = MultiClassBayesNode(
    "SEX",
    [],
    cpt_sex_group
)

In [399]:
diabetes_node = MultiClassBayesNode(
    "DIABETES",
    [],
    cpt_diabetes_group
)

In [400]:
# define the bayesian network
classification_final = BayesNet([
    age_node,
    sex_node,
    obesity_node,
    diabetes_node,
    pneumonia_node,
    icu_node,
    classification_node
])

In [401]:
print(classification_final)

BayesNet([('AGE_GROUP', ''), ('SEX', ''), ('OBESITY', ''), ('DIABETES', ''), ('PNEUMONIA', 'AGE_GROUP'), ('ICU', 'PNEUMONIA'), ('CLASIFFICATION_FINAL', 'SEX OBESITY DIABETES ICU')])


In [402]:
print("CPT for CLASIFFICATION_FINAL:", cpt_classification)
print("CPT for ICU:", cpt_icu)
print("CPT for PNEUMONIA:", cpt_pneumonia)

CPT for CLASIFFICATION_FINAL: {(1, 1, 1, 1): {1: 0.6524822695035462, 0: 0.3475177304964539}, (1, 1, 1, 2): {0: 0.5135019593872462, 1: 0.4864980406127538}, (1, 1, 2, 1): {0: 0.328998699609883, 1: 0.6710013003901171}, (1, 1, 2, 2): {0: 0.5990560540729518, 1: 0.40094394592704813}, (1, 1, 98, 1): {0: 0.42857142857142855, 1: 0.5714285714285714}, (1, 1, 98, 2): {0: 0.475, 1: 0.525}, (1, 2, 1, 1): {1: 0.6375121477162293, 0: 0.3624878522837707}, (1, 2, 1, 2): {0: 0.5469492144683961, 1: 0.45305078553160394}, (1, 2, 2, 1): {1: 0.5160128102481986, 0: 0.48398718975180144}, (1, 2, 2, 2): {1: 0.31796763023376756, 0: 0.6820323697662325}, (1, 2, 98, 1): {1: 0.5, 0: 0.5}, (1, 2, 98, 2): {0: 0.6729857819905213, 1: 0.32701421800947866}, (1, 98, 1, 1): {1: 1.0}, (1, 98, 1, 2): {0: 0.49122807017543857, 1: 0.5087719298245614}, (1, 98, 2, 1): {1: 0.5, 0: 0.5}, (1, 98, 2, 2): {1: 0.3333333333333333, 0: 0.6666666666666666}, (1, 98, 98, 1): {0: 0.43333333333333335, 1: 0.5666666666666667}, (1, 98, 98, 2): {0: 0.

# Evaluate the Model

In [403]:
def predict_bayes_net(bn, evidence, query_var):
    """
    Predict the most likely value of a query variable given evidence using the Bayesian Network.
    
    Args:
        bn: Bayesian network.
        evidence: Dictionary of evidence variables and their values.
        query_var: Variable to predict.
    
    Returns:
        The most likely value of the query variable.
    """
    result = enumeration_ask(query_var, evidence, bn)
    return max(result.prob, key=lambda k: result.prob[k])

In [404]:
def evaluate_bayes_net(bn, test_data, query_var):
    """
    Evaluate the Bayesian Network on a test dataset.
    
    Args:
        bn: Bayesian Network (BayesNet instance).
        test_data: Test dataset (pandas DataFrame).
        query_var: The target variable to predict.
    
    Returns:
        Accuracy of the predictions as a float.
    """
    correct = 0  # Count of correct predictions

    for _, row in test_data.iterrows():
        # Build evidence dictionary from test row
        evidence = {
            "AGE_GROUP": row["AGE_GROUP"],
            "SEX": row["SEX"],
            "OBESITY": row["OBESITY"],
            "DIABETES": row["DIABETES"],
            "PNEUMONIA": row["PNEUMONIA"],
            "ICU": row["ICU"]
        }

        # Predict the target variable
        prediction = predict_bayes_net(bn, evidence, query_var)
        
        
        # Check if the prediction matches the actual value
        if prediction == row[query_var]:
            correct += 1

    # Calculate accuracy
    accuracy = correct / len(test_data)
    return accuracy

In [405]:
# Evaluate the Bayesian Network on the test dataset
query_var = "CLASIFFICATION_FINAL"
accuracy = evaluate_bayes_net(classification_final, test_data, query_var)

print(f"Accuracy of Bayesian Network: {accuracy:.2%}")

Accuracy of Bayesian Network: 63.22%


In [406]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

def evaluate_bayes_net(bn, test_data, query_var):
    """
    Evaluate the Bayesian Network on a test dataset and compute various metrics.

    Args:
        bn: Bayesian Network (BayesNet instance).
        test_data: Test dataset (pandas DataFrame).
        query_var: The target variable to predict.

    Returns:
        metrics: A dictionary containing accuracy, and prints out confusion matrix
                 and classification report (precision, recall, f1).
    """
    y_true = []
    y_pred = []

    for _, row in test_data.iterrows():
        # Build evidence dictionary from test row
        # Note: Adjust the evidence set according to what you want to condition on.
        evidence = {
            "AGE_GROUP": row["AGE_GROUP"],
            "SEX": row["SEX"],
            "OBESITY": row["OBESITY"],
            "DIABETES": row["DIABETES"],
            "PNEUMONIA": row["PNEUMONIA"],
            "ICU": row["ICU"]
        }

        # Predict the target variable
        prediction = predict_bayes_net(bn, evidence, query_var)

        y_true.append(row[query_var])
        y_pred.append(prediction)

    # Calculate accuracy
    acc = accuracy_score(y_true, y_pred)
    cm = confusion_matrix(y_true, y_pred)
    report = classification_report(y_true, y_pred, zero_division=0)

    print("Confusion Matrix:\n", cm)
    print("\nClassification Report:\n", report)

    metrics = {
        "accuracy": acc,
    }

    return metrics

In [407]:
metrics = evaluate_bayes_net(classification_final, test_data, "CLASIFFICATION_FINAL")
print("Accuracy:", metrics["accuracy"])

Confusion Matrix:
 [[124064   7255]
 [ 69877   8519]]

Classification Report:
               precision    recall  f1-score   support

           0       0.64      0.94      0.76    131319
           1       0.54      0.11      0.18     78396

    accuracy                           0.63    209715
   macro avg       0.59      0.53      0.47    209715
weighted avg       0.60      0.63      0.55    209715

Accuracy: 0.6322056123787044
